In [ ]:
# This script implements a simple trading strategy based on a
# 200-day moving average and a 2-period RSI.

# Install the required libraries if you haven't already:
# pip install yfinance pandas matplotlib numpy

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def calculate_rsi(data, period):
    """Calculates the Relative Strength Index (RSI)."""
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def main():
    """Main function to run the trading strategy."""
    # --- Configuration ---
    # You can change the ticker to any stock symbol you want.
    ticker_symbol = "BBNI.JK"
    start_date = "2010-01-01"
    end_date = pd.Timestamp.today().strftime('%Y-%m-%d')
    initial_capital = 10000.0

    print(f"Downloading daily data for {ticker_symbol} from {start_date} to {end_date}...")
    try:
        df = yf.download(ticker_symbol, start=start_date, end=end_date)
        if df.empty:
            print("Error: No data downloaded. Check the ticker symbol and date range.")
            return
    except Exception as e:
        print(f"An error occurred while downloading data: {e}")
        return

    print("Data downloaded successfully.")

    # --- Strategy Rules & Indicators ---
    # 1. 200-day Moving Average (SMA)
    df['SMA_200'] = df['Close'].rolling(window=200).mean()

    # 2. Relative Strength Index (RSI) with a period of 2 as requested.
    # Note: A period of 2 is very short and will result in a highly volatile signal.
    # A more common period is 14.
    df['RSI'] = calculate_rsi(df['Close'], period=2)

    # --- Backtesting the Strategy ---
    # Initialize variables
    in_position = False
    shares = 0
    equity = [initial_capital]
    cash = initial_capital
    entry_dates = []
    exit_dates = []

    # Iterate through the data to find buy/sell signals
    # We start after the first 200 days to allow the SMA to be calculated.
    for i in range(200, len(df) - 1): # -1 to avoid index error on 'next day'
        current_day = df.iloc[i]
        next_day = df.iloc[i+1]

        # BUY SIGNAL:
        # Closing price is above the 200 SMA AND RSI is below 10.
        # Fixed ValueError by explicitly using .item() to extract scalar values for all comparisons.
        if (not np.isnan(current_day['SMA_200']).item()) and (not np.isnan(current_day['RSI']).item()) and (current_day['Close'].item() > current_day['SMA_200'].item()) and (current_day['RSI'].item() < 10) and not in_position:
            # Buy at today's close
            shares_to_buy = cash // current_day['Close'].item()
            if shares_to_buy > 0:
                shares += shares_to_buy
                cash -= shares_to_buy * current_day['Close'].item()
                in_position = True
                entry_dates.append(current_day.name)
                print(f"BUY Signal on {current_day.name.date()}: Executing buy at {current_day.name.date()} close (${current_day['Close'].item():.2f})")

        # SELL SIGNAL:
        # Today's closing price is greater than yesterday's high.
        # Fixed the same ValueError here by using .item() on both sides of the comparison.
        if in_position and (current_day['Close'].item() > df.iloc[i-1]['High'].item()):
            # Sell at today's close
            cash += shares * current_day['Close'].item()
            shares = 0
            in_position = False
            exit_dates.append(current_day.name)
            print(f"SELL Signal on {current_day.name.date()}: Executing sell at {current_day.name.date()} close (${current_day['Close'].item():.2f})")

        # Update equity at the end of each day
        current_equity = cash + shares * current_day['Close'].item()
        equity.append(current_equity)

    # --- Plotting the Results ---
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    fig.suptitle('Trading Strategy Backtest', fontsize=16)

    # Plot 1: Stock Price & Indicators
    ax1.plot(df.index, df['Close'], label='Closing Price', color='blue', alpha=0.8)
    ax1.plot(df.index, df['SMA_200'], label='200-Day SMA', color='orange', linestyle='--')
    ax1.set_title(f'{ticker_symbol} Stock Price and 200-Day SMA')
    ax1.set_ylabel('Price ($)')
    ax1.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax1.legend()

    # Add markers for buy and sell points
    buy_prices = [df.loc[date]['Close'].item() for date in entry_dates]
    sell_prices = [df.loc[date]['Close'].item() for date in exit_dates]
    ax1.scatter(entry_dates, buy_prices, marker='^', color='green', s=100, label='Buy Signal')
    ax1.scatter(exit_dates, sell_prices, marker='v', color='red', s=100, label='Sell Signal')

    # Add watermark to the first chart
    ax1.text(1.0, 0.0, 'backtested by Handiko', transform=ax1.transAxes, fontsize=12, color='darkgray', ha='right', va='bottom', fontweight='bold')

    # Plot 2: Equity Growth and Stock Price on a logarithmic scale
    # Create a secondary y-axis to plot the closing price
    ax2_2 = ax2.twinx()

    # Plot Equity Growth
    equity_df = pd.Series(equity, index=df.index[200:])
    ax2.plot(equity_df.index, equity_df.values, label='Equity Growth', color='purple')
    ax2.axhline(y=initial_capital, color='gray', linestyle='--', label='Initial Capital')
    ax2.set_ylabel('Equity ($)')
    ax2.set_yscale('log')
    ax2.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax2.legend(loc='upper left')

    # Plot Closing Price on the secondary axis
    ax2_2.plot(df.index, df['Close'], label='Closing Price', color='blue', alpha=0.8)
    ax2_2.set_ylabel('Closing Price ($)')
    ax2_2.set_yscale('log')
    ax2_2.legend(loc='upper right')

    ax2.set_title(f'Equity Growth vs. Stock Price for {ticker_symbol} (Log Scale)')
    ax2.set_xlabel('Date')

    # Add watermark to the second chart
    ax2.text(1.0, 0.0, 'backtested by Handiko', transform=ax2.transAxes, fontsize=12, color='darkgray', ha='right', va='bottom', fontweight='bold')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # Save the plot to a file
    plot_filename = f'{ticker_symbol}_trading_strategy_plot.png'
    plt.savefig(plot_filename)
    print(f"\nPlot saved to {plot_filename}")

    # --- Final Results ---
    final_equity = equity[-1]
    total_return = (final_equity - initial_capital) / initial_capital * 100
    print(f"\n--- Final Results ---")
    print(f"Initial Capital: ${initial_capital:.2f}")
    print(f"Final Equity: ${final_equity:.2f}")
    print(f"Total Return: {total_return:.2f}%")

if __name__ == "__main__":
    main()
